# An Exploration in the Success of Starbucks

### Karol Paya, Joshua Paul Cohen, Michelle Yiv

[Source: Macrotrends.net](https://www.macrotrends.net/stocks/charts/SBUX/starbucks/shares-outstanding)

[Source: fred.stlouisfed.org](https://fred.stlouisfed.org/series/CPIAUCSL)

change color, additional annotations, annotations

![Workflow](final_proj_workflow.png)

# Import Data

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import requests
from io import BytesIO
from PIL import Image

# Load the starbucks dataset
url = "data/starbucks_revenue_data.xlsx"
revenue = pd.read_excel(url)

# load the inflation data set
inflation_url = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CPIAUCSL&scale=left&cosd=1947-01-01&coed=2024-08-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2024-09-12&revision_date=2024-09-12&nd=1947-01-01"
inflation = pd.read_csv(inflation_url, parse_dates=["DATE"]).rename(columns = {"CPIAUCSL": "index", "DATE": "date"})

# Load stocks data set
stocks=pd.read_excel('data/starbucks_shares.xlsx')

# Cleaning Data

In [ ]:
# Starbucks
# convert date column to datetime
revenue['date'] = pd.to_datetime(revenue['date'])

In [ ]:
# Inflation
# Change index column from object to numeric, remove NA
inflation["index"] = pd.to_numeric(inflation["index"], errors='coerce')
inflation = inflation.dropna()

In [ ]:
# stocks data set was already clean

# Exploring Data

In [ ]:
# Preview the starbucks data set
print("Most Recent 5 Quarters\n",revenue.head(5))

print("Summary Stats:\n", revenue.describe())

print("NA Values:\n", revenue.isnull().sum())

In [ ]:
# Preview the inflation data set
print("Most Recent 5 Quarters\n",inflation.tail(5))

print("Summary Stats:\n", inflation.describe())

print("NA Values:\n",inflation.isnull().sum())

In [ ]:
# Preview the stocks data set
print("Most Recent 5 Shares\n",stocks.head(5))
desc = stocks.describe()
print(desc)

print("NA Values:\n", stocks.isnull().sum())

# Filtering Data

In [ ]:
# Inflation
# create a new column filtering after 2009 because stocks data started after 2009
inflation_2009_plus = inflation.loc[(inflation['date'] >= '2009-01-01')]

# Sorting Data

In [ ]:
# Starbucks
# Sort data by most millions of usd
revenue_sorted = revenue.sort_values('millions_of_usd', ascending=False)
print("Most Productive Quarters\n", revenue_sorted.head(20))

# Transforming Data

In [ ]:
# Starbucks
# Add a month column to categorize by season
revenue['month'] = revenue['date'].dt.month

# Use an if statement to assign seasons
def season (month):
    if month == 12:
        return "Winter"
    elif month == 3:
        return "Spring"
    elif month == 6:
        return "Summer"
    else:
        return "Fall"
    
# add a season column
revenue['season'] = revenue['month'].map(season)

In [ ]:
# join data sets

revenue_stocks = pd.merge(revenue, stocks, on='date', how='inner')
revenue_stocks

# Grouping Data

In [ ]:
# Find average earnings by season
revenue_season = revenue.groupby('season').median().reset_index()
print("Median Earnings by Season \n", revenue_season)

# Aggregating Data

In [ ]:
# Starbucks
# find the average by year

# add a year column
revenue['year'] = revenue['date'].dt.year

# find the average by year
revenue_average = revenue.groupby(['year'], as_index=False)['millions_of_usd'].mean()
print("Average Earnings by Year\n", revenue_average)


In [ ]:
# Inflation
# find the percent change by quarter
inflation_change = inflation_2009_plus.diff().round(2)
inflation_change["date"] = inflation_2009_plus["date"]
inflation_change = inflation_change.rename(columns = {"index": "change_quarterly_percent"})

# Visualizing Data

In [ ]:
# Starbucks
# plot earnings by season

sns.barplot(
    data=revenue_season,
     x='season', y='millions_of_usd')
plt.title('Average Earnings by Season')
plt.xlabel('Year')
plt.ylabel('Millions of USD')
plt.show()


# plot earnings by year

sns.lineplot(
    data=revenue_average,
    x='year', y='millions_of_usd')
plt.title('Average Earnings by Year')
plt.xlabel('Year')
plt.ylabel('Millions of USD')
plt.show()



In [ ]:
# Inflation
# plot of inflation index by year
sns.lineplot(data = inflation_2009_plus, x = "date", y = "index")
plt.title('Inflation index by Year')
plt.xlabel('Year')
plt.ylabel('Index')
plt.show()

# plot of Change in Quarterly Percent
sns.lineplot(data = inflation_change, x = "date", y = "change_quarterly_percent")
plt.title('Rate of Change per Quarter')
plt.xlabel('Year')
plt.ylabel('Change in Quarterly Percent')
plt.show()

In [ ]:
# plot stocks plot
stocks.plot(x='date',y='Starbucks Quarterly Shares (Millions of Shares)', legend=False)
plt.title('Starbucks Stock Shares')
plt.xlabel('Year')
plt.ylabel('Quartely Shares (millions)')
plt.show()


In [ ]:
## Overlay Graphs

# create empty figure

fig, ax1 = plt.subplots(figsize=(10,10))

# create axis 1
ax1.plot(stocks['date'],stocks['Starbucks Quarterly Shares (Millions of Shares)'], 'steelblue', label='stocks')
ax1.tick_params(axis='y')
plt.ylabel('Change in Quarterly Percent')

# create axis 2
ax2=ax1.twinx()
ax2.plot(inflation_change['date'],  inflation_change['change_quarterly_percent'], 'coral', label='inflation')
ax2.tick_params(axis='y')
plt.ylabel('Rate of Change per Quarter')

# legend
lines, labels = ax1.get_legend_handles_labels() 
lines2, labels2 = ax2.get_legend_handles_labels() 
ax1.legend(lines + lines2, labels + labels2, loc='upper right')

# labels
plt.title('Stocks vs Inflation Change')
plt.xlabel('Year')
plt.show()

In [ ]:
## Inflation vs Revenue

# create empty figure
fig, ax1 = plt.subplots(figsize=(10,10))

# create axis 1
ax1.plot(revenue['date'],revenue['millions_of_usd'], 'steelblue', label='Millions USD')
ax1.tick_params(axis='y')
plt.ylabel('Millions USD')

# create axis 2
ax2=ax1.twinx()
ax2.plot(inflation_change['date'],  inflation_change['change_quarterly_percent'], 'coral', label='inflation')
ax2.tick_params(axis='y')
plt.ylabel('Rate of Change per Quarter')

# create legend
lines, labels = ax1.get_legend_handles_labels() 
lines2, labels2 = ax2.get_legend_handles_labels() 
ax1.legend(lines + lines2, labels + labels2, loc='upper left')

# labels
plt.title('Inflation vs Revenue')
plt.xlabel('Year')
plt.show()